In [2]:
# biblioteki
from src.dataclass import TextDataSet, ImageDataSet
from transformers import ViTImageProcessor, AutoModelForImageClassification
import torch
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import  DataLoader
from torchvision.transforms import transforms, ToPILImage, ToTensor

In [3]:
# model z hugingface
extractor = ViTImageProcessor.from_pretrained("DunnBC22/dit-base-Business_Documents_Classified_v2")

model = AutoModelForImageClassification.from_pretrained("DunnBC22/dit-base-Business_Documents_Classified_v2")

In [9]:
# ładowanie danych
image_data_set = ImageDataSet('../datasets/train_set', extractor)

dataloader = DataLoader(
    image_data_set,
    batch_size=256,
    shuffle=True,
)

In [5]:
# zmiana głowicy
model.classifier = torch.nn.Linear(in_features=model.classifier.in_features, out_features=21)

In [6]:
# zamrożenie parametrów

for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

In [8]:
# trening głowicy
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
num_epochs = 10
losses = []


for epoch in tqdm(range(num_epochs)):
    running_loss = 0
    for train_features, train_labels in tqdm(dataloader):
        optimizer.zero_grad()

        outputs = model(train_features).logits

        train_labels = train_labels.type(torch.float32)

        loss = criterion(outputs, train_labels)
        loss.backward()

        optimizer.step()
        print(loss.item())
        running_loss += loss.item()

    al = running_loss / len(image_data_set)
    losses.append(al)

plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

  0%|          | 1/1357 [00:04<1:37:57,  4.33s/it]

0.6711298823356628



  0%|          | 2/1357 [00:07<1:28:13,  3.91s/it]

1.4052138328552246



  0%|          | 3/1357 [00:11<1:24:21,  3.74s/it]

...

1.2871912717819214



  1%|          | 7/1357 [00:26<1:24:39,  3.76s/it]

0.6594662666320801


  0%|          | 0/10 [00:27<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
for image, label in image_data_set:
    # Pextractor
    inputs = extractor(images=image, return_tensors="pt")

    # inferencja
    logits = model(**inputs).logits
    probs = torch.nn.functional.softmax(logits, dim=-1)

    # klasfikacja
    predicted_class = torch.argmax(probs).item()

    print("Przewidywana klasa:", predicted_class, "Prawdziwa klasa:", label)
